# Test LiteLLM

In [2]:
from litellm import completion

In [14]:
system_prompt = "You are a university-level physics professor teaching a graduate course in Physics."
#system_prompt = "You are kindergarden teacher who introduce elementary physics concetps to a group of 5 year old in a language that they can understand."
prompt = "Explain nuclear fusion in 5 sentences or less."

In [15]:
list_of_models = {
    "Gemini": "gemini/gemini-1.5-flash",
    "Groq": "groq/llama3-70b-8192",
    "Mistral": "mistral/mistral-large-2411",
    #"OpenRouter": "openrouter/deepseek/deepseek-r1:free",
    "OpenRouter": "openrouter/meta-llama/llama-3.1-405b-instruct:free",
}

In [16]:
for provider, model in list_of_models.items():
    print("="*25)
    print(f"Using model {model} from {provider}:\n")
    try:
        response_l = completion(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
        )
        print(response_l['choices'][0].message.content)
    except Exception as e:
        print(f"The model failed, raising the following exception: {e}")

Using model gemini/gemini-1.5-flash from Gemini:


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

The model failed, raising the following exception: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}

Using model groq/llama3-70b-8192 from Groq:

Nuclear fusion is the process by which atomic nuclei combine to form a heavier nucleus, releasing energy in the process. This occurs at extremely high temperatures, typically above 15 million degrees Celsius, where the nuclei have sufficient energy to overcome their mutual repulsion. The most commonly discussed fusion reaction is the deuterium-tritium (D-T) reaction, where deuterium (a heavy isotope of hydrogen) combines with tritium (a rare isotope of hydrogen) to form helium and a h

# Test fallback

In [27]:
ll_models = list(list_of_models.values())
response_l = completion(
    model=ll_models[-1],
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    fallbacks=ll_models[:-1],
)
print(response_l['choices'][0].message.content)

Nuclear fusion is a process where two or more atomic nuclei combine to form a single, heavier nucleus, releasing a vast amount of energy in the process. This process is the opposite of nuclear fission, where an atomic nucleus splits into lighter nuclei. The most commonly discussed fusion reaction is the deuterium-tritium (D-T) reaction, where a deuterium nucleus (one proton and one neutron) combines with a tritium nucleus (one proton and two neutrons) to form a helium nucleus and a high-energy neutron. The energy released in fusion reactions is several orders of magnitude greater than that released in chemical reactions, making it a promising source of clean and sustainable energy. However, achieving controlled nuclear fusion has proven to be a significant technical challenge due to the extremely high temperatures and pressures required to initiate and sustain the reaction.


In [26]:
response_l

ModelResponse(id='chatcmpl-63ac749c-900a-40b1-8c28-d0a1b1be1aac', created=1738209284, model='gemini-1.5-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Nuclear fusion is the process where two light atomic nuclei combine to form a heavier nucleus, releasing a tremendous amount of energy.  This energy release stems from the conversion of a small amount of mass into energy, as described by Einstein's famous equation, E=mc².  The process requires overcoming the electrostatic repulsion between the positively charged nuclei, typically through extremely high temperatures and pressures.  Fusion powers the sun and other stars.  Harnessing fusion energy on Earth remains a significant technological challenge, but offers a potentially clean and virtually limitless energy source.\n", role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=110, prompt_tokens=26, total_tokens=136, compl

# Compare Gemini and Groq models

## Gemini

In [2]:
import os
import google.generativeai as genai
from groq import Groq


genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

/usr/local/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [46]:
model_name = "gemini-1.5-flash"
param = {"temperature": 0.0, "top_p": 0.1}
prompt = "Describe OpenAI in a few sentences."

In [47]:
model = genai.GenerativeModel(model_name)
response_g = model.generate_content(
    prompt,
    generation_config = genai.GenerationConfig(**param)
)
response_g.text

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

In [48]:
# https://docs.litellm.ai/docs/providers/gemini
response_l = completion(
    model=f"gemini/{model_name}",
    messages=[{"role": "user", "content": prompt}],
    **param
)
response_l['choices'][0].message.content

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

### Check reproducibility in the case temperature=0.0 and top_p < 1.0
In the case top_p = 1.0, I would still get some stochasticity, probably since temperature=0.0 is approximated algorithmically (using eps)

In [36]:
model = genai.GenerativeModel(model_name)
response_g = model.generate_content(
    prompt,
    generation_config = genai.GenerationConfig(**param)
)
response_g.text

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

In [37]:
model = genai.GenerativeModel(model_name)
response_g = model.generate_content(
    prompt,
    generation_config = genai.GenerationConfig(**param)
)
response_g.text

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

In [38]:
response_l = completion(
    model=f"gemini/{model_name}",
    messages=[{"role": "user", "content": prompt}],
    **param
)
response_l['choices'][0].message.content

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

## Compare Groq

In [49]:
model_name = "llama-3.3-70b-versatile"
param = {"temperature": 0.0, "top_p": 0.1}
prompt = "Describe OpenAI in a few sentences."

In [50]:
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt,}],
    model=model_name,
    **param
)
print(chat_completion.choices[0].message.content)

OpenAI is a leading artificial intelligence research organization that aims to develop and promote friendly AI that benefits humanity. Founded in 2015, OpenAI is known for its cutting-edge AI models, including language models like GPT-3 and DALL-E, which have achieved state-of-the-art results in various AI tasks. The organization is committed to transparency, open-source development, and responsible AI practices, and its research and technologies have far-reaching implications for fields such as natural language processing, computer vision, and robotics.


In [51]:
# https://docs.litellm.ai/docs/providers/groq
response_l = completion(
    model=f"groq/{model_name}",
    messages=[{"role": "user", "content": prompt}],
    **param
)
response_l['choices'][0].message.content

'OpenAI is a leading artificial intelligence research organization that aims to develop and promote friendly AI that benefits humanity. Founded in 2015, OpenAI is known for its cutting-edge AI models, including language models like GPT-3 and DALL-E, which have achieved state-of-the-art results in various AI tasks. The organization is committed to transparency, open-source development, and responsible AI practices, and its research and technologies have far-reaching implications for fields such as natural language processing, computer vision, and robotics.'